### Imports 

In [ ]:
import requests
from bs4 import BeautifulSoup

1. Send the request for accesing first webpage with opinions about product 

In [ ]:
with open('./cookie.json','r', encoding="UTF-8") as jf:
    headers = json.load(jf)

In [ ]:
url = "https://www.ceneo.pl/17230358#tab=reviews"
response = requests.get(url)


2. if response is ok, parse HTML page content into DOM structure

In [ ]:
if response.status_code == 200:
    page_dom = BeautifulSoup(response.text, "html.parser")
    

3. Extract opinions and their components from DOM structure 

In [ ]:
opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
print(type(page_dom))
print(type(opinions))
print(len(opinions))
opinion = opinions.pop(0)
print(type(opinion))
